<a href="https://colab.research.google.com/github/Acion2/cripto/blob/main/rail_fence_comentado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
r

In [ ]:
import hashlib  # Biblioteca para funções de hash (como SHA-256 usado no PBKDF2)
import os       # Biblioteca para funcionalidades do sistema operacional, como gerar dados aleatórios
from google.colab import files  # Módulo específico do Google Colab para interação com arquivos (upload/download)
import tkinter as tk   # Importa a biblioteca Tkinter para criar interfaces gráficas (usada para a janela de seleção de arquivos)
from tkinter import filedialog  # Submódulo do Tkinter para abrir diálogos de arquivo (selecionar arquivo)
from tkinter import messagebox  # Submódulo do Tkinter para exibir caixas de mensagens (erro, sucesso)

def gerar_chave_salt(senha):
    """
    Gera um salt aleatório e uma chave derivada usando PBKDF2 (Password-Based Key Derivation Function 2).
    PBKDF2 é uma função de derivação de chaves resistente a ataques de força bruta e de dicionário.

    Args:
        senha (str): A senha fornecida pelo usuário.

    Returns:
        tuple: Uma tupla contendo a chave derivada (bytes) e o salt (bytes).
    """
    salt = os.urandom(16)  # Gera 16 bytes aleatórios para usar como salt. O salt é único para cada chave.
    chave = hashlib.pbkdf2_hmac('sha256', senha.encode('utf-8'), salt, 100000)
    # Aplica a função PBKDF2_HMAC com o algoritmo SHA-256 na senha (codificada para bytes),
    # usando o salt gerado e repetindo o processo 100.000 vezes para aumentar a segurança.
    # Quanto maior o número de iterações, mais difícil é quebrar a chave por força bruta.
    return chave, salt

def cifrar_rail_fence(texto, trilhos):
    """
    Cifra um texto usando a Cifra de Trilho de Ferro (Rail Fence Cipher).
    Esta é uma cifra de transposição que embaralha a ordem das letras.

    Args:
        texto (str): O texto plano a ser cifrado.
        trilhos (int): O número de "trilhos" a serem usados na cifra.

    Returns:
        str: O texto cifrado.
    """
    matriz = [['' for _ in range(len(texto))] for _ in range(trilhos)]
    # Cria uma matriz vazia (lista de listas) com o número de trilhos e o comprimento do texto.
    direcao = 1   # 1 para baixo, -1 para cima (controla o movimento vertical na matriz)
    linha, coluna = 0, 0  # Inicializa a linha e a coluna para escrever na matriz.

    for char in texto:
        matriz[linha][coluna] = char  # Escreve o caractere atual na posição da matriz.
        coluna += 1  # Move para a próxima coluna.
        linha += direcao  # Move para a próxima linha (para cima ou para baixo).

        # Inverte a direção se atingir o topo ou a base dos trilhos.
        if linha == trilhos - 1 or linha == 0:
            direcao *= -1

    texto_cifrado = ''.join(''.join(row) for row in matriz)
    # Junta todas as linhas da matriz em uma única string para obter o texto cifrado.
    return texto_cifrado

def decifrar_rail_fence(texto_cifrado, trilhos):
    """
    Decifra um texto cifrado usando a Cifra de Trilho de Ferro.

    Args:
        texto_cifrado (str): O texto cifrado a ser decifrado.
        trilhos (int): O número de "trilhos" usados para cifrar o texto.

    Returns:
        str: O texto plano decifrado.
    """
    matriz = [['' for _ in range(len(texto_cifrado))] for _ in range(trilhos)]
    # Cria uma matriz vazia com as dimensões corretas.
    direcao = 1
    linha, coluna = 0, 0

    # Preenche a matriz com marcadores '*' para identificar a ordem das letras no padrão zigue-zague.
    for _ in range(len(texto_cifrado)):
        matriz[linha][coluna] = '*'
        coluna += 1
        linha += direcao
        if linha == trilhos - 1 or linha == 0:
            direcao *= -1

    # Preenche a matriz com as letras do texto cifrado na ordem correta.
    indice = 0
    texto_decifrado_matriz = [['' for _ in range(len(texto_cifrado))] for _ in range(trilhos)]
    for i in range(trilhos):
        for j in range(len(texto_cifrado)):
            if matriz[i][j] == '*':
                texto_decifrado_matriz[i][j] = texto_cifrado[indice]
                indice += 1

    # Lê a matriz novamente no padrão zigue-zague para obter o texto plano original.
    texto_decifrado = ''
    linha, coluna = 0, 0
    direcao = 1
    for _ in range(len(texto_cifrado)):
        texto_decifrado += texto_decifrado_matriz[linha][coluna]
        coluna += 1
        linha += direcao
        if linha == trilhos - 1 or linha == 0:
            direcao *= -1

    return texto_decifrado

def cifrar_arquivo(nome_arquivo, senha, trilhos):
    """
    Cifra o conteúdo de um arquivo texto usando a Cifra de Trilho de Ferro e PBKDF2.

    Args:
        nome_arquivo (str): O nome do arquivo a ser cifrado.
        senha (str): A senha para cifrar o arquivo.
        trilhos (int): O número de trilhos para a cifra.

    Returns:
        tuple: Uma tupla contendo o nome do arquivo cifrado e o texto cifrado (ou None, None em caso de erro).
    """
    try:
        with open(nome_arquivo, 'r') as arquivo:
            texto_plano = arquivo.read()
    except FileNotFoundError:
        print(f"Erro: Arquivo '{nome_arquivo}' não encontrado.")
        return None, None

    chave, salt = gerar_chave_salt(senha)
    texto_cifrado = cifrar_rail_fence(texto_plano, trilhos)

    nome_arquivo_base = os.path.splitext(os.path.basename(nome_arquivo))[0]
    nome_arquivo_cifrado = f"{nome_arquivo_base}_cifrado.txt"

    try:
        with open(nome_arquivo_cifrado, 'w') as arquivo_cifrado:
            # Armazena o salt (em hexadecimal), o número de trilhos e o texto cifrado no arquivo.
            # O '$' é usado como um delimitador para facilitar a leitura durante a decifragem.
            arquivo_cifrado.write(f"{salt.hex()}${trilhos}${texto_cifrado}")
        print(f"Arquivo '{nome_arquivo}' cifrado com sucesso para '{nome_arquivo_cifrado}'.")
        return nome_arquivo_cifrado, texto_cifrado
    except Exception as e:
        print(f"Erro ao escrever no arquivo cifrado: {e}")
        return None, None

def decifrar_arquivo(nome_arquivo_cifrado, senha):
    """
    Decifra o conteúdo de um arquivo cifrado com Rail Fence e PBKDF2.

    Args:
        nome_arquivo_cifrado (str): O nome do arquivo cifrado a ser decifrado.
        senha (str): A senha para decifrar o arquivo.

    Returns:
        tuple: Uma tupla contendo o nome do arquivo decifrado e o texto decifrado (ou None, None em caso de erro).
    """
    try:
        with open(nome_arquivo_cifrado, 'r') as arquivo_cifrado:
            conteudo_cifrado = arquivo_cifrado.read().split('$', 2)
            salt_hex = conteudo_cifrado[0]
            trilhos = int(conteudo_cifrado[1])
            texto_cifrado = conteudo_cifrado[2]
    except FileNotFoundError:
        print(f"Erro: Arquivo '{nome_arquivo_cifrado}' não encontrado.")
        return None, None
    except ValueError:
        print("Erro: Formato do arquivo cifrado inválido.")
        return None, None

    salt = bytes.fromhex(salt_hex)  # Converte o salt de hexadecimal de volta para bytes.
    chave_derivada = hashlib.pbkdf2_hmac('sha256', senha.encode('utf-8'), salt, 100000)
    # Recalcula a chave derivada usando a senha fornecida e o salt lido do arquivo.
    # Embora a chave derivada seja calculada, neste código simplificado,
    # a autenticidade da senha não é verificada explicitamente antes da decifragem.
    # Em um sistema real, você compararia essa chave derivada com uma chave armazenada.

    nome_arquivo_base = os.path.splitext(os.path.basename(nome_arquivo_cifrado))[0].replace("_cifrado", "")
    nome_arquivo_decifrado = f"{nome_arquivo_base}_decifrado.txt"

    texto_decifrado = decifrar_rail_fence(texto_cifrado, trilhos)
    try:
        with open(nome_arquivo_decifrado, 'w') as arquivo_decifrado:
            arquivo_decifrado.write(texto_decifrado)
        print(f"Arquivo '{nome_arquivo_cifrado}' decifrado com sucesso para '{nome_arquivo_decifrado}'.")
        return nome_arquivo_decifrado, texto_decifrado
    except Exception as e:
        print(f"Erro ao escrever no arquivo decifrado: {e}")
        return None, None

if __name__ == "__main__":
    while True:
        acao = input("Digite 'cifrar' para cifrar um arquivo, 'decifrar' para decifrar (ou 'sair' para encerrar): ").lower()
        if acao == 'cifrar':
            print("Digite o nome do arquivo a ser cifrado (certifique-se de que ele foi carregado no Colab):")
            nome_arquivo = input()
            senha = input("Digite a senha para cifrar: ")
            while True:
                try:
                    trilhos = int(input("Digite o número de 'trilhos' para a cifra: "))
                    if trilhos > 1:
                        break
                    else:
                        print("O número de trilhos deve ser maior que 1.")
                except ValueError:
                    print("Por favor, digite um número inteiro válido para os trilhos.")
            nome_arquivo_cifrado, texto_cifrado = cifrar_arquivo(nome_arquivo, senha, trilhos)
            if nome_arquivo_cifrado:
                print(f"Arquivo cifrado salvo como '{nome_arquivo_cifrado}'.")
                # Opcional: fazer o download do arquivo cifrado para sua máquina local
                # files.download(nome_arquivo_cifrado)
        elif acao == 'decifrar':
            print("Digite o nome do arquivo a ser decifrado (certifique-se de que ele foi carregado no Colab):")
            nome_arquivo_cifrado = input()
            senha = input("Digite a senha para decifrar: ")
            nome_arquivo_decifrado, texto_decifrado = decifrar_arquivo(nome_arquivo_cifrado, senha)
            if nome_arquivo_decifrado:
                print(f"Arquivo decifrado salvo como '{nome_arquivo_decifrado}'.")
                # Opcional: fazer o download do arquivo decifrado para sua máquina local
                # files.download(nome_arquivo_decifrado)
        elif acao == 'sair':
            break
        else:
            print("Ação inválida. Por favor, digite 'cifrar', 'decifrar' ou 'sair'.")